# Machine learning for solid solutions (Li2TiS3)

This notebook is divided into these sections:
- [LTS dataset](#lts)
    - write CRYSTAL input files
    - read CRYSTAL output files
- [Descriptos](#descriptors)
- [Machine learning](#ml)
    - linear regression

In [1]:
import os
import copy
import json
import itertools
import shutil as sh
from pathlib import Path
import numpy as np
import pandas as pd

from CRYSTALpytools.crystal_io import Crystal_output, Crystal_input, Crystal_density, Crystal_gui
from CRYSTALpytools.convert import cry_gui2pmg, cry_out2pmg
from CRYSTALpytools.utils import view_pmg

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer, PointGroupAnalyzer

from ase.visualize import view

from dscribe.descriptors import CoulombMatrix

from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,15)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# <a id='lts'>LTS dataset</a>

In [4]:
# New atom
new_atom = 'Li'

# Read the confcount output
cry_output = Crystal_output().read_cry_output('data/crystal/lts/lts_confcount.out')
cry_output.get_config_analysis()

# Read the initial structure (before substitution)
original_structure_gui =  Crystal_gui().read_cry_gui('data/crystal/lts/lts_confcount.gui')
original_structure = cry_gui2pmg(original_structure_gui)

structures_lts = []
li_atoms = []
ti_atoms = []
for j,substitutions in enumerate(cry_output.atom_type2):
    new_structure = original_structure.copy()
    for i in substitutions:
        new_structure.replace(i-1,new_atom)
    structures_lts.append(new_structure)
    ti_atoms.append((np.array(cry_output.atom_type1[j])-1).tolist())
    li_atoms.append((np.array(cry_output.atom_type2[j])-1).tolist())

structures = copy.deepcopy(structures_lts)

In [5]:
len(structures)

4023

# <a id='descriptors'>Descriptors</a>

@alex: Build the dscribe descriptors for the LTS structures

# <a id='ml'>Machine learning</a>